In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Aug 16 17:14:42 2019
 
@author: mjofre
"""
###############################################################################
# Script to analyze and compute the Secure Key Rate of the BB84+Decoy-state transmission.
###############################################################################
import numpy as np
 
def TransmissionBB84DecoyStateComputeSKRcontinuousphase(SourcePulseRate,DecoyStatesRates,DecoyStatesPhotPulse,AttenuationdBforDistance,SystemDetectorPerformanceArray,DVQKDprotocol):
    # H.-K. Lo, X. Ma, and K. Chen, “Decoy State Quantum Key Distribution,” Phys. Rev. Lett., vol. 94, no. 23, p. 230504, Jun. 2005, doi: 10.1103/PhysRevLett.94.230504.
    
    RateAlice=SourcePulseRate
    Nlen=len(AttenuationdBforDistance)
    alphaLink=0.1 # 0.1 dB/Km link loss free-space attenuation#0.2dB/Km for 1550nm telecom fiber
    Distance=AttenuationdBforDistance/alphaLink*1000.0#np.linspace(0,350000,len(Measured_EMu))
    #Distance_plot=Distance*1e-3
    
    Y0=SystemDetectorPerformanceArray[0]
    e0=SystemDetectorPerformanceArray[1]
    transmittanceBob=SystemDetectorPerformanceArray[2]
    attDetectors=SystemDetectorPerformanceArray[3]

    MuRate=DecoyStatesRates[0] # Rate of signal states sent
    Nu1Rate=DecoyStatesRates[1] # Rate of decoy 1 states sent
    Nu2Rate=DecoyStatesRates[2] # Rate of decoy 2 states sent
    
    if ((MuRate+Nu1Rate+Nu2Rate)!=1.0):
        print('MuRate+Nu1Rate+Nu2Rate: '+str(MuRate+Nu1Rate+Nu2Rate))
        print('Error in the settings rates of the signal and decoy rates (MuRate+Nu1Rate+Nu2Rate)!=1.0')
        return
    
    fe=1.16 # bidirectional error correction factor
    
    Mu=DecoyStatesPhotPulse[0]#0.5
    Nu1=DecoyStatesPhotPulse[1]#Mu*np.power(10,(-4.65/10.0))
    Nu2=DecoyStatesPhotPulse[2]#Mu*np.power(10,(-14.76/10.0))
    
    attLink=np.power(10,(-(alphaLink/10.0)*(Distance/1000.0))) # Channel transmittance
    attBob=attDetectors*transmittanceBob # denote the transmittance in Bob's side: internal transmittance of optical components and detector efficiency
    att=attLink*attBob#*(1-1/50); # Overall transmission and detection efficiency between Alice and Bob
    
    RateOutputAlice_signal=RateAlice*Mu # Rate of signals states sent out by Alice
    RKR_plot=RateOutputAlice_signal*att # RKR due to signal states
    
    # Yield of a i-photon pulse
    Y1 = Y0+(1-(1-att)**1)
    Y2 = Y0+(1-(1-att)**2)
    
    # Fung, C.-H.F., Tamaki, K., Lo, H.-K.: Performance of two quantum-key-distribution protocols. Phys. Rev. A 73(1), 012337 (2006)
    # Ma, X., Qi, B., Zhao, Y., Lo, H.-K.: Practical decoy state for quantum key distribution. Phys. Rev. A 72(1), 012326 (2005)
    # Gain of states
    QMu = Y0+1-np.exp(-att*Mu) # Gain of the signal state. Equivalent to Y0 if Mu=0
    QNu1 = Y0+1-np.exp(-att*Nu1) # Gain of the decoy state. Equivalent to Y0 if Nu1=0
    QNu2 = Y0+1-np.exp(-att*Nu2) # Gain of the vacuum state. Equivalent to Y0 if Nu2=0

    edetector=0#1.0-cos(atan(0.05))**2#0.01; # Probability that a photon hits the wrong detector. It has to due with misalignment and finite PER of polarization splitters
    EMu = (e0*Y0+edetector*(1-np.exp(-att*Mu)))/QMu # QBER for the signal state
    ENu1 = (e0*Y0+edetector*(1-np.exp(-att*Nu1)))/QNu1 # QBER for the decoy state
    ENu2 = (e0*Y0+edetector*(1-np.exp(-att*Nu2)))/QNu2 # QBER for the vacuum state
        
    ## Computations theoretical
    #print('Nu1: '+str(Nu1))
    #print('Nu2: '+str(Nu2))
    #print('QNu1: '+str(QNu1))
    #print('QNu2: '+str(QNu2))
    
    Y0LowerBound = (Nu1*QNu2*np.exp(Nu2)-Nu2*QNu1*np.exp(Nu1))/(Nu1-Nu2) # Yield.
    if (np.sum(Y0LowerBound<0.0)>0):
        Y0LowerBound[Y0LowerBound<0.0]=0.0
    
    Y1LowerBound = (np.exp(-Mu))/(Mu*Nu1-Mu*Nu2-Nu1**2+Nu2**2)*(QNu1*np.exp(Nu1)-QNu2*np.exp(Nu2)-(Nu1**2-Nu2**2)/(Mu**2)*(QMu*np.exp(Mu)-Y0LowerBound)) # Yield of single photon pulses
    if (np.sum(Y1LowerBound<0.0)>0):
        Y1LowerBound[Y1LowerBound<0.0]=0.0
    
    Y2LowerBound = Y2# approx #(np.exp(-Mu))/(Mu*Nu1-Mu*Nu2-Nu1**2+Nu2**2)*(QNu1*np.exp(Nu1)-QNu2*np.exp(Nu2)-(Nu1**2-Nu2**2)/(Mu**2)*(QMu*np.exp(Mu)-Y0LowerBound)) # Yield of two-photon pulses
    if (np.sum(Y2LowerBound<0.0)>0):
        Y2LowerBound[Y2LowerBound<0.0]=0.0
    
    Q0LowerBound = Mu*np.exp(-Mu)*Y0LowerBound # Gain of the vacuum-photon pulses.    
    Q1LowerBound = Mu*np.exp(-Mu)*Y1LowerBound # Gain of the single photon pulses.
    Q2LowerBound = ((Mu**2)/2)*np.exp(-Mu)*Y2LowerBound # Gain of the two-photon pulses. # # Ma, X., Qi, B., Zhao, Y., Lo, H.-K.: Practical decoy state for quantum key distribution. Phys. Rev. A 72(1), 012326 (2005)

    e0Bound=e0*np.ones((int(len(Y0LowerBound))),dtype=np.float32)
    e1UpperBound = (EMu*QMu*np.exp(Mu)-e0Bound*Y0LowerBound)/(Q1LowerBound*np.exp(Mu)) # Error rate of single photon pulses
    e2UpperBound = (EMu*QMu*np.exp(Mu)-e0Bound*Y0LowerBound)/(Q2LowerBound*np.exp(Mu))#1-(1-(EMu*QMu*np.exp(Mu)-e0Bound*Y0LowerBound)/(Q2LowerBound*np.exp(Mu)))**2 # Error rate of two-photon pulses
    
    if (np.sum(e1UpperBound<0.0)>0):
        e1UpperBound[e1UpperBound<0.0]=0.0
    
    if (np.sum(e1UpperBound>1.0)>0):
        e1UpperBound[e1UpperBound>1.0]=1.0
    
    if (np.sum(e2UpperBound<0.0)>0):
        e2UpperBound[e2UpperBound<0.0]=0.0
    
    if (np.sum(e2UpperBound>1.0)>0):
        e2UpperBound[e2UpperBound>1.0]=1.0
    
    QBER=e1UpperBound # QBER due only to single-photon. In general the bit error ratio of n-photon signals being e_n=Y_0/(2*Y_n)
    
    H2EMu=np.zeros(int(len(EMu)),dtype=np.float32)
    for iIter in range(0,len(EMu),1):
        if (EMu[iIter]<=0.0):
            H2EMu[iIter]=0.0
        else:
            H2EMu[iIter] = -EMu[iIter]*np.log2(EMu[iIter])-(1.0-EMu[iIter])*np.log2(1.0-EMu[iIter])
    
    H2e0Upper=np.zeros(int(len(e0Bound)),dtype=np.float32)
    for iIter in range(0,len(e0Bound),1):
        if (e0Bound[iIter]<=0.0):
            H2e0Upper[iIter]=0.0
        else:
            H2e0Upper[iIter] = -e0Bound[iIter]*np.log2(e0Bound[iIter])-(1.0-e0Bound[iIter])*np.log2(1.0-e0Bound[iIter])
    
    H2e1Upper=np.zeros(int(len(e1UpperBound)),dtype=np.float32)
    for iIter in range(0,len(e1UpperBound),1):
        if (e1UpperBound[iIter]<=0.0):
            H2e1Upper[iIter]=0.0
        else:
            H2e1Upper[iIter] = -e1UpperBound[iIter]*np.log2(e1UpperBound[iIter])-(1.0-e1UpperBound[iIter])*np.log2(1.0-e1UpperBound[iIter])
    
    H2e2Upper=np.zeros(int(len(e2UpperBound)),dtype=np.float32)
    for iIter in range(0,len(e2UpperBound),1):
        if (e2UpperBound[iIter]<=0.0):
            H2e2Upper[iIter]=0.0
        else:
            H2e2Upper[iIter] = -e2UpperBound[iIter]*np.log2(e2UpperBound[iIter])-(1.0-e2UpperBound[iIter])*np.log2(1.0-e2UpperBound[iIter])
    
    # Lower Secure Key Rate
    if (DVQKDprotocol=='SARG04+decoy'): # One-photon and 2-photon pulses contribute
      CoincidenceSameBasePrepMeas=0.25 # For SARG04 it is 0.25.
      KeyBitRateLowerBond = CoincidenceSameBasePrepMeas*(-QMu*fe*H2EMu + Q1LowerBound*(1.0-H2e1Upper)+Q2LowerBound*(1.0-H2e2Upper))*RateOutputAlice_signal
    else: # for instance 'BB84+decoy' only one-photon pulses contribute
      CoincidenceSameBasePrepMeas=0.5 # For BB84 it is 0.5; for SARG04 it is 0.25.
      KeyBitRateLowerBond = CoincidenceSameBasePrepMeas*(-QMu*fe*H2EMu + Q1LowerBound*(1.0-H2e1Upper))*RateOutputAlice_signal#CoincidenceSameBasePrepMeas*(-QMu*fe*H2EMu + Q0LowerBound*(1.0-H2e0Upper) + Q1LowerBound*(1.0-H2e1Upper))*RateOutputAlice_signal
    
    if (np.sum(KeyBitRateLowerBond<0.0)>0):
        KeyBitRateLowerBond[KeyBitRateLowerBond<0.0]=0.0
        
    # Remove glitches
    for iIter in range(1,int(Nlen-2),1):
        if (np.logical_and(KeyBitRateLowerBond[iIter-1]<=0.0,KeyBitRateLowerBond[iIter+1]<=0.0) or np.logical_and(KeyBitRateLowerBond[iIter-2]<=0.0,KeyBitRateLowerBond[iIter+2]<=0.0)):
            KeyBitRateLowerBond[iIter]=0.0
    
    if (np.sum(KeyBitRateLowerBond<=0.0)>0):
        QBER[KeyBitRateLowerBond<=0.0]=0.5
    
    if (np.sum(QBER>=0.5)>0): # Not to be displayed in the image
        QBER[QBER>=0.5]=0.0
        
    return attLink,RKR_plot,KeyBitRateLowerBond,QBER